### Custom JSON Encoding using JSONEncoder

In the previous video, we saw how we were able to provide custom encodings using the `default` argument of the `dump`/`dumps` function.

But how does Python know how to encode the "standard" types, such as `str`, `int`, `float`, `list`, `dict`, etc?

It uses a special class - `JSONEncoder`.

This class supports the following encodings (see Python docs: https://docs.python.org/3/library/json.html#json.JSONEncoder)

|Python |JSON  |
|:----|:---|
| `dict` | object `{...}`|
| `list`, `tuple` | array `[...]` |
| `str`  | string `"..."`|
| `int`, `float` | number |
| `int` or `float` `Enums` | number |
| `bool` | `true` or `false` |
| `None` | `null` |

Anything beyond those Python types and we end up with a `TypeError` exception.

We can see how this class encodes objects by calling an instance of it directly:

In [1]:
import json

default_encoder = json.JSONEncoder()
default_encoder.encode([1, 2, 3])

'[1, 2, 3]'

And for non-supported objects:

In [2]:
default_encoder.encode(1+1j)

TypeError: Object of type 'complex' is not JSON serializable

We can actually extend this `JSONEncoder` class and override the `default` method. We can then add in support for whatever type we want to use, and pass any other types to the parent class to handle (either serialize the data or raise a `TypeError` exception). 

Let's just see a simple example first:

In [3]:
import json
from datetime import datetime

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            super().default(arg)

In [4]:
custom_encoder = CustomJSONEncoder()

In [5]:
custom_encoder.encode(True)

'true'

In [6]:
custom_encoder.encode(datetime.utcnow())

'"2018-12-29T22:27:19.863377"'

And we can now use this custom encoder by specifying it when we use `dump`/`dumps`:

In [7]:
json.dumps(dict(name='test', time=datetime.utcnow()), cls=CustomJSONEncoder)

'{"name": "test", "time": "2018-12-29T22:27:20.135841"}'

One thing to note is that for both the `default` approach, and the `cls` approach, our method / encoder will only be used for types that Python cannot already serialize on its own (strings, integers, lists, etc).

In [8]:
def custom_encoder(arg):
    print('Custom encoder called...')
    if isinstance(arg, str):
        return f'some string: {arg}'

Here we want to "override" `dumps` default encoding behavior for strings:

In [9]:
json.dumps({'name': 'Python'}, default=custom_encoder)

'{"name": "Python"}'

As you can see, we cannot do that - because the argument is a "recognized" type (`str`), Python does not even call our `custom_encoder` function.

And the same happens when we override the `default` method in our custom `JSONEncoder` class.

Let's look at the signature for `dumps`:

In [10]:
help(json.dumps)

Help on function dumps in module json:

dumps(obj, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)
    Serialize ``obj`` to a JSON formatted ``str``.
    
    If ``skipkeys`` is true then ``dict`` keys that are not basic types
    (``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
    instead of raising a ``TypeError``.
    
    If ``ensure_ascii`` is false, then the return value can contain non-ASCII
    characters if they appear in strings contained in ``obj``. Otherwise, all
    such characters are escaped in JSON strings.
    
    If ``check_circular`` is false, then the circular reference check
    for container types will be skipped and a circular reference will
    result in an ``OverflowError`` (or worse).
    
    If ``allow_nan`` is false, then it will be a ``ValueError`` to
    serialize out of range ``float`` values (``nan``, ``inf``, ``-inf``) in
    stric

And let's see the signature for `JSONEncoder`:

In [11]:
help(json.JSONEncoder)

Help on class JSONEncoder in module json.encoder:

class JSONEncoder(builtins.object)
 |  Extensible JSON <http://json.org> encoder for Python data structures.
 |  
 |  Supports the following objects and types by default:
 |  
 |  +-------------------+---------------+
 |  | Python            | JSON          |
 |  +===================+===============+
 |  | dict              | object        |
 |  +-------------------+---------------+
 |  | list, tuple       | array         |
 |  +-------------------+---------------+
 |  | str               | string        |
 |  +-------------------+---------------+
 |  | int, float        | number        |
 |  +-------------------+---------------+
 |  | True              | true          |
 |  +-------------------+---------------+
 |  | False             | false         |
 |  +-------------------+---------------+
 |  | None              | null          |
 |  +-------------------+---------------+
 |  
 |  To extend this to recognize other objects, subclas

Here we are particularly interested in the `__init__` method signature:

 `__init__(self, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, sort_keys=False, indent=None, separators=None, default=None)`

`dumps(obj, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)`

In [12]:
d = {
    'a': float('inf'),
    'b': [1, 2, 3]
}

In [13]:
d

{'a': inf, 'b': [1, 2, 3]}

In [14]:
type(d['a'])

float

As you can see, that float is a special type of float - it represents + infinity.

Let's see if Python can encode that:

In [15]:
json.dumps(d)

'{"a": Infinity, "b": [1, 2, 3]}'

Yes, it does - but notice the output, `Infinity`. Technically this is not JSON... (see https://tools.ietf.org/html/rfc4627 Section 2.4)

So, if we want to be strict about this, and ensure we are not trying to serialize a value such as infinity, we would do this instead:

In [16]:
json.dumps(d, allow_nan=False)

ValueError: Out of range float values are not JSON compliant

And we get the desired result.

What about trying to encode an invalid key (from JSON's perspective)::

In [17]:
d = {10: "int", 10.5: "float", 1+1j: "complex"}

In [18]:
d

{10: 'int', 10.5: 'float', (1+1j): 'complex'}

These are all valid Python dictionary keys, but what happens with JSON encoding?

In [19]:
json.dumps(d)

TypeError: keys must be a string

As you can see we get an exception. We may want to simply ignore that exception and not include the offending key/value pair in our serialization:

In [20]:
json.dumps(d, skipkeys=True)

'{"10": "int", "10.5": "float"}'

And now we no longer get an exception, and the complex key was simply skipped.

We can even change how the serialization is rendered (which of course means we may no longer have actual JSON):

In [21]:
d = {
    'name': 'Python',
    'age': 27,
    'created_by': 'Guido van Rossum',
    'list': [1, 2, 3]
}

In [22]:
json.dumps(d)

'{"name": "Python", "age": 27, "created_by": "Guido van Rossum", "list": [1, 2, 3]}'

In [23]:
print(json.dumps(d, indent='---', separators=('', ' = ')))

{
---"name" = "Python"
---"age" = 27
---"created_by" = "Guido van Rossum"
---"list" = [
------1
------2
------3
---]
}


We can use this by the way, to create more compact JSON strings (uses less bytes):

In [24]:
print(json.dumps(d))

{"name": "Python", "age": 27, "created_by": "Guido van Rossum", "list": [1, 2, 3]}


vs

In [25]:
print(json.dumps(d, separators=(',', ':')))

{"name":"Python","age":27,"created_by":"Guido van Rossum","list":[1,2,3]}


As you can see, all the whitespace is eliminated. For transmitting large JSON objects, that can make a (relatively small) difference in making the JSON more compact.

So, if we want to consistently use the same values for all those tweaks, we have to consistently remember to set the arguments correctly in the `dump`/`dumps` functions.

Instead, we could create a custom JSONEncoder class that pre-sets all these things, and just use that encoder - simpler than remembering all those arguments and their correct values:

In [26]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(skipkeys=True, 
                         allow_nan=False, 
                         indent='---', 
                         separators=('', ' = ')
                        )
        
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)

In [27]:
d = {
    'time': datetime.utcnow(),
    1+1j: "complex",
    'name': 'Python'
}

In [28]:
print(json.dumps(d, cls=CustomEncoder))

{
---"time" = "2018-12-29T22:27:26.689488"
---"name" = "Python"
}


Another thing I want to point out is that with both these methods we are not limited in what we emit as our JSON serialization.

For example, for a `datetime` object, we may want to emit not only the ISO formatted date, but maybe some additional fields, all nested within a JSON object:

In [29]:
class CustomEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            obj = dict(
                datatype="datetime",
                iso=arg.isoformat(),
                date=arg.date().isoformat(),
                time=arg.time().isoformat(),
                year=arg.year,
                month=arg.month,
                day=arg.day,
                hour=arg.hour,
                minutes=arg.minute,
                seconds=arg.second
            )
            return obj
        else:
            return super().default(arg)

In [30]:
d = {
    'time': datetime.utcnow(),
    'message': 'Testing...'
}

In [31]:
print(json.dumps(d, cls=CustomEncoder, indent=2))

{
  "time": {
    "datatype": "datetime",
    "iso": "2018-12-29T22:27:27.668208",
    "date": "2018-12-29",
    "time": "22:27:27.668208",
    "year": 2018,
    "month": 12,
    "day": 29,
    "hour": 22,
    "minutes": 27,
    "seconds": 27
  },
  "message": "Testing..."
}
